In [2]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.model_settings import ModelSettings

from pydantic import BaseModel

from dotenv import load_dotenv
import asyncio
import os

from IPython.display import display, Markdown
from pprint import pprint

import requests

load_dotenv(override=True)

True

# OpenAI Tools
- `WebSearchTool`: search the web
- `FileSearchTool`: retrieve from OpenAI vector stores
- `ComputerTool`: automate computer tasks

# Agents to Make
- Search Agent: search online
- Planner Agent: come up with searches based on user's query
- Report Agent: make a report
- Push Agent: send a notification to the user's phone with a summary

In [3]:
gemini_client = AsyncOpenAI(base_url=os.environ['GEMINI_BASE_URL'], api_key=os.environ['GEMINI_API_KEY'])
gemini_model = OpenAIChatCompletionsModel(model='gemini-2.5-flash', openai_client=gemini_client)

#### Dummy Tools (Avoid Hosted Tools)

In [ ]:
# custom dummy tools to avoid hosted tools / pricing

@function_tool
async def web_search_tool(query: str):
    """Surf the web based on the given query
    
    Args:
        query: The user's query
    """

    return """
    Search Result 1: AutoGen
    Search Result 2: LangChain
    Search Result 3: LangGraph
    Search Result 4: CrewAI
    Search Result 5: OpenAI Agents SDK
    """

#### Agent 1 - Search Agent

In [9]:
INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 words. \
Capture the main points. Write succint, no need to have complete sentences or good grammar. This will be \
consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do \
not include any additional commentary other than the summary itself."""

search_agent = Agent(
    name='Search Agent',
    instructions=INSTRUCTIONS,
    # tools=[WebSearchTool(search_context_size='low')],
    tools=[web_search_tool],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice='required'),
)

In [10]:
message = 'What are the most popular and successful AI Agent frameworks in July 2025'
with trace('Search'):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


As of 2024, prominent AI Agent frameworks include AutoGen, LangChain, LangGraph, CrewAI, and OpenAI Agents SDK. These frameworks are widely adopted for building and managing AI agents, offering tools and abstractions to streamline development.

LangChain and LangGraph are well-established for their comprehensive features in orchestrating complex AI workflows, including chaining different models and tools. AutoGen, on the other hand, focuses on enabling multi-agent conversations and collaborations.

CrewAI is gaining traction for its capabilities in defining and managing AI agent teams, while the OpenAI Agents SDK provides direct integration with OpenAI's models for agent creation. These frameworks represent the current leading solutions for developing intelligent agent-based applications.

OPENAI_API_KEY is not set, skipping trace export


#### Agent 2 - Planner Agent

In [11]:
HOW_MANY_SEARCHES = 5
INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Ouptut {HOW_MANY_SEARCHES} terms to query for."

# structured outputs
class WebSearchItem(BaseModel):
    reason: str  # force the model to tell us its thinking
    "Your reasoning for why this search is important to the query"

    query: str
    "The search term to use for the web search"

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query"""

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model=gemini_model,
    output_type=WebSearchPlan,
)

In [12]:
message = "What are the most popular and successful AI Agent frameworks in July 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To establish a baseline of currently popular AI agent frameworks, which are likely to continue their success or evolve.', query='most popular AI agent frameworks 2024'), WebSearchItem(reason='To identify newer frameworks or significant trends expected to gain traction by July 2025.', query='emerging AI agent framework trends projections 2025'), WebSearchItem(reason="To understand the criteria and metrics that define 'success' for AI agent frameworks, such as community support, performance, and enterprise adoption.", query='AI agent framework success factors adoption rates'), WebSearchItem(reason='To find information on significant R&D efforts from leading organizations or academia that might lead to influential frameworks by 2025.', query='top AI agent research and development initiatives'), WebSearchItem(reason='To look for industry analyses, expert opinions, or foresight reports discussing the evolution and potential leaders in AI agent f

OPENAI_API_KEY is not set, skipping trace export


In [13]:
for search in result.final_output.searches:
    print(search.reason)
    print(search.query)
    print()

To establish a baseline of currently popular AI agent frameworks, which are likely to continue their success or evolve.
most popular AI agent frameworks 2024

To identify newer frameworks or significant trends expected to gain traction by July 2025.
emerging AI agent framework trends projections 2025

To understand the criteria and metrics that define 'success' for AI agent frameworks, such as community support, performance, and enterprise adoption.
AI agent framework success factors adoption rates

To find information on significant R&D efforts from leading organizations or academia that might lead to influential frameworks by 2025.
top AI agent research and development initiatives

To look for industry analyses, expert opinions, or foresight reports discussing the evolution and potential leaders in AI agent frameworks towards mid-2025.
future of AI agents and framework predictions



#### Agent 3 - Writer Agent

In [14]:
INSTRUCTIONS = """
You are a senior researcher tasked with writing a cohesive report for a research query.
You will be provided with the original query and some initial research done by a research assistant.

You should first come up with an outline for the report that describes the structure and flow of the report.
Then, generate the report and return that as your final output.

The final output should be in markdown format and it should be length and detailed.
Aim for 5-10 pages of content, at least 1000 words.
"""

class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings"""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

writer_agent = Agent(
    name='WriterAgent',
    instructions=INSTRUCTIONS,
    model=gemini_model,
    output_type=ReportData
)

#### Agent 4 - Push Notification Agent

In [ ]:
# https://pushover.net

pushover_user = ...
pushover_token = ...
pushover_url = ...

@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

# dummy tool to avoid tools with pricing
@function_tool
def push_tool(message: str):
    """Send a push notification with this brief message"""
    print(f'Dummy tool called to send push notification with message: {message}')
    return {"status": "success"}

In [ ]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you received."""

push_agent = Agent(
    name='push_agent',
    instructions=INSTRUCTIONS,
    tools=[push_tool],
    model=gemini_model,
    model_settings=ModelSettings(tool_choice=['required'])
)